In [5]:
import pandas as pd
import os

# Base relative path from current notebook
base_path = "../../results/pipeline results/5884paras_598qna/copies/"

# File paths
simple_rag_path = os.path.join(base_path, "simple_rag_qna_results_GPU_version - Copy.csv")
lqr_path = os.path.join(base_path, "LQR_processed_results - Copy.csv")
mod_lqr_path = os.path.join(base_path, "modLQR_processed_results - Copy.csv")

# Read CSVs
simple_rag_df = pd.read_csv(simple_rag_path)
lqr_df = pd.read_csv(lqr_path)
mod_lqr_df = pd.read_csv(mod_lqr_path)

# Rename columns for consistency
simple_rag_df = simple_rag_df.rename(columns={
    "retriever_time": "retrieval_time",
    "generator_time": "generation_time"
})

# Calculate average times
simple_rag_avg = simple_rag_df[["retrieval_time", "generation_time", "total_time"]].mean()

lqr_avg = lqr_df[[
    "classification_time",
    "decomposition_time",
    "retrieval_time",
    "generation_time",
    "total_time",
    "total_time_without_classification"
]].mean()

mod_lqr_avg = mod_lqr_df[[
    "classification_time",
    "decomposition_time",
    "retrieval_time",
    "generation_time",
    "total_time",
    "total_time_without_classification"
]].mean()

# Combine into a summary table
summary_df = pd.DataFrame({
    "Simple RAG": simple_rag_avg,
    "LQR RAG": lqr_avg,
    "Mod LQR RAG": mod_lqr_avg
})

# Display
summary_df = summary_df.rename_axis("Metric").reset_index()
print(summary_df)


                              Metric  Simple RAG    LQR RAG  Mod LQR RAG
0                classification_time         NaN   3.484323     5.794626
1                 decomposition_time         NaN   4.564814     6.040382
2                    generation_time    6.886000   2.004932     5.179983
3                     retrieval_time    0.056868   4.366314     7.407531
4                         total_time    6.942945  14.420383    24.422523
5  total_time_without_classification         NaN  10.936060    19.173777


In [6]:

base_save_path = "../../results/scores_results/times/"

output_path = os.path.join(base_save_path, "average_times_summary.csv")
summary_df.to_csv(output_path, index=False)

print(f"Summary saved to: {output_path}")

Summary saved to: ../../results/scores_results/times/average_times_summary.csv


In [7]:
import pandas as pd
import os

# Base relative paths
base_read_path = "../../results/pipeline results/5884paras_598qna/copies/"
base_save_path = "../../results/scores_results/times/"

# File paths
simple_rag_path = os.path.join(base_read_path, "simple_rag_qna_results_GPU_version - Copy.csv")
lqr_path = os.path.join(base_read_path, "LQR_processed_results - Copy.csv")
mod_lqr_path = os.path.join(base_read_path, "modLQR_processed_results - Copy.csv")

# Read CSVs
simple_rag_df = pd.read_csv(simple_rag_path)
lqr_df = pd.read_csv(lqr_path)
mod_lqr_df = pd.read_csv(mod_lqr_path)

# Rename for consistency
simple_rag_df = simple_rag_df.rename(columns={
    "retriever_time": "retrieval_time",
    "generator_time": "generation_time"
})

# Slice data after index 98
simple_rag_df = simple_rag_df.iloc[99:]
lqr_df = lqr_df.iloc[99:]
mod_lqr_df = mod_lqr_df.iloc[99:]

# Compute averages
simple_rag_avg = simple_rag_df[["retrieval_time", "generation_time", "total_time"]].mean()

lqr_avg = lqr_df[[
    "classification_time",
    "decomposition_time",
    "retrieval_time",
    "generation_time",
    "total_time",
    "total_time_without_classification"
]].mean()

mod_lqr_avg = mod_lqr_df[[
    "classification_time",
    "decomposition_time",
    "retrieval_time",
    "generation_time",
    "total_time",
    "total_time_without_classification"
]].mean()

# Create summary DataFrame
summary_df = pd.DataFrame({
    "Simple RAG": simple_rag_avg,
    "LQR RAG": lqr_avg,
    "Mod LQR RAG": mod_lqr_avg
})

summary_df = summary_df.rename_axis("Metric").reset_index()

# Save to CSV
output_path = os.path.join(base_save_path, "average_times_summary_after_98.csv")
summary_df.to_csv(output_path, index=False)

print(f"Summary for records after index 98 saved to: {output_path}")


Summary for records after index 98 saved to: ../../results/scores_results/times/average_times_summary_after_98.csv


In [8]:
summary_df

,Metric,Simple RAG,LQR RAG,Mod LQR RAG
0,classification_time,NaN,3.489935,5.776418
1,decomposition_time,NaN,5.459756,7.226446
2,generation_time,6.886346,2.062940,5.334306
3,retrieval_time,0.056214,5.209255,8.848235
4,total_time,6.942635,16.221886,27.185405
5,total_time_without_classification,NaN,12.731951,21.408987
